In [7]:
import scipy.io.idl as idl
import numpy as np
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [8]:
from scipy.spatial import ConvexHull, Delaunay
from scipy.interpolate import griddata

In [9]:
from pyvisfile.vtk import write_structured_grid
from pytools.obj_array import make_obj_array

# Define Functions

In [10]:
z_planes = [0.249, 0.302, 0.357, 0.416]

In [22]:
def read_idl(quantity, data_path='../../comprehensive_3d_plot/output/2016-04-07/'):
    r"""
    Read idl files for all planes.
    """
    idl_ending = '.sav'
    z249_file = data_path + quantity + '_z249' + idl_ending
    z302_file = data_path + quantity + '_z302' + idl_ending
    z357_file = data_path + quantity + '_z357' + idl_ending
    z416_file = data_path + quantity + '_z416' + idl_ending
    
    z249_measurements = idl.readsav(z249_file)
    z302_measurements = idl.readsav(z302_file)
    z357_measurements = idl.readsav(z357_file)
    z416_measurements = idl.readsav(z416_file)
    
    measurements = {0.249: z249_measurements, 
                    0.302: z302_measurements, 
                    0.357: z357_measurements, 
                    0.416: z416_measurements}

    return measurements

In [34]:
def read_points_from_measurement_dict(measurement_dict, time_point, z_planes):
    x_points = np.empty((0))
    y_points = np.empty((0))
    z_points = np.empty((0))
    values = np.empty((0))
    for z_plane in z_planes:
        plane_measurements = measurement_dict[z_plane]
        x_points = np.append(x_points, plane_measurements['x_out'])
        y_points = np.append(y_points, plane_measurements['y_out'])
        z_points = np.append(z_points, np.ones(plane_measurements['x_out'].size)*z_plane)
        values = np.append(values, plane_measurements['a_out'][time_point])
    
    
    points = [x_points, y_points, z_points]
    points = np.asarray(points)
    points = np.swapaxes(points, 0, 1)
    return points, values

In [24]:
def bounded_grid(bounds, spatial_increment=0.003):
    
    (x_min, x_max), (y_min, y_max), (z_min, z_max) = bounds
    
    x_coord = np.linspace(x_min, x_max, np.ceil((x_max-x_min)/spatial_increment))
    y_coord = np.linspace(y_min, y_max, np.ceil((y_max-y_min)/spatial_increment))
    z_coord = np.linspace(z_min, z_max, np.ceil((z_max-z_min)/spatial_increment))
    
    sizes = map(np.size, [x_coord, y_coord, z_coord])

    mesh = np.meshgrid(x_coord, y_coord, z_coord, indexing='ij')
    
    grid_points = np.dstack(map(np.ravel, mesh))[0]
    
    return grid_points, sizes

In [25]:
def interpolate_vector(grid_points, vector_points, vector_values):
    interpolated_data_x = griddata(vector_points[0], vector_values[0], grid_points)
    interpolated_data_y = griddata(vector_points[1], vector_values[1], grid_points)
    interpolated_data_z = griddata(vector_points[2], vector_values[2], grid_points)
    return [interpolated_data_x, interpolated_data_y, interpolated_data_z]

In [26]:
def interpolate_scalar(grid_points, scalar_points, scalar_values):
    interpolated_data = griddata(scalar_points, scalar_values, grid_points)
    return interpolated_data

In [27]:
def add_vacuum_field(field, vacuum_field=0.02):
    field[2] = field[2] + vacuum_field
    return field

In [28]:
def prepare_mesh(grid_points, sizes):
    return grid_points.swapaxes(0,1).reshape((3, sizes[0], sizes[1], sizes[2]))

In [29]:
def prepare_vector(vector, sizes):
    vtk_vector_x = np.resize(vector[0], sizes)
    vtk_vector_y = np.resize(vector[1], sizes)
    vtk_vector_z = np.resize(vector[2], sizes)

    vtk_vector_x = np.expand_dims(vtk_vector_x, 0)
    vtk_vector_y = np.expand_dims(vtk_vector_y, 0)
    vtk_vector_z = np.expand_dims(vtk_vector_z, 0)

    vtk_vector = make_obj_array([vtk_vector_x, vtk_vector_y, vtk_vector_z])
    return vtk_vector

In [30]:
def prepare_scalar(scalar, sizes):
    vtk_scalar = np.resize(scalar, sizes)
    vtk_scalar = np.expand_dims(scalar, 0)
    return vtk_scalar

# Interpolate B field

In [390]:
x_min, x_max = -0.027, 0.022
y_min, y_max = -0.021, 0.0295 
z_min, z_max = 0.249, 0.416

b_bounds = ((x_min, x_max), (y_min, y_max), (z_min, z_max)) 

In [391]:
bx_measurements = read_idl('bx')
by_measurements = read_idl('by')
bz_measurements = read_idl('bz')

In [392]:
direction_measurements = [bx_measurements, by_measurements, bz_measurements] 

In [395]:
for time_point in xrange(21):
    print 'time_point %i' % time_point
    points = []
    values = []
    for measurements in direction_measurements:
        points_direction, values_direction = read_points_from_measurement_dict(measurements, time_point, z_planes)
        points.append(points_direction)
        values.append(values_direction)
    grid_points, sizes = bounded_grid(b_bounds, 0.003)
    interpolated_vector = interpolate_vector(grid_points, points, values)
    interpolated_vector = add_vacuum_field(interpolated_vector)

    assert np.sum(np.isnan(interpolated_vector[0])) == 0
    assert np.sum(np.isnan(interpolated_vector[1])) == 0
    assert np.sum(np.isnan(interpolated_vector[2])) == 0

    
    
    vtk_grid = prepare_mesh(grid_points, sizes)
    vtk_vector = prepare_vector(interpolated_vector, sizes)

    output_path = '../output/2016-06-14/'
    write_structured_grid(output_path + 
                          'b_' + str(time_point).zfill(4) + '.vts', vtk_grid, point_data=[('B', vtk_vector)])

time_point 0
time_point 1
time_point 2
time_point 3
time_point 4
time_point 5
time_point 6
time_point 7
time_point 8
time_point 9
time_point 10
time_point 11
time_point 12
time_point 13
time_point 14
time_point 15
time_point 16
time_point 17
time_point 18
time_point 19
time_point 20


# Interpolate Te & n

In [31]:
x_min, x_max = -0.022, 0.018
y_min, y_max = -0.021, 0.0255 
z_min, z_max = 0.249, 0.416

te_bounds = ((x_min, x_max), (y_min, y_max), (z_min, z_max)) 

In [32]:
te_measurements = read_idl('te')
n_measurements = read_idl('n')

In [35]:
for time_point in xrange(21):
    print 'time_point %i' % time_point
    te_points, te_values = read_points_from_measurement_dict(te_measurements, time_point, z_planes)
    grid_points, sizes = bounded_grid(te_bounds, 0.003)
    te_interpolated_scalar = interpolate_scalar(grid_points, te_points, te_values)
    
    assert np.sum(np.isnan(te_interpolated_scalar)) == 0
    
    vtk_grid = prepare_mesh(grid_points, sizes)
    te_vtk_scalar = prepare_scalar(te_interpolated_scalar, sizes)
    
    
    print 'time_point %i' % time_point
    n_points, n_values = read_points_from_measurement_dict(n_measurements, time_point, z_planes)
    n_interpolated_scalar = interpolate_scalar(grid_points, n_points, n_values)
    
    assert np.sum(np.isnan(n_interpolated_scalar)) == 0
    
    n_vtk_scalar = prepare_scalar(n_interpolated_scalar, sizes)
    
    
    output_path = '../output/2016-06-14/'
    write_structured_grid(output_path + 
                          'p_' + str(time_point).zfill(4) + '.vts', vtk_grid, point_data=[('Te', te_vtk_scalar),
                                                                                          ('n', n_vtk_scalar)])

time_point 0
time_point 0
time_point 1
time_point 1
time_point 2
time_point 2
time_point 3
time_point 3
time_point 4
time_point 4
time_point 5
time_point 5
time_point 6
time_point 6
time_point 7
time_point 7
time_point 8
time_point 8
time_point 9
time_point 9
time_point 10
time_point 10
time_point 11
time_point 11
time_point 12
time_point 12
time_point 13
time_point 13
time_point 14
time_point 14
time_point 15
time_point 15
time_point 16
time_point 16
time_point 17
time_point 17
time_point 18
time_point 18
time_point 19
time_point 19
time_point 20
time_point 20
